In [1]:
import pandas as pd
from g4f.client import Client

In [2]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    加載 CSV 文件並處理錯誤行。
    """
    try:
        return pd.read_csv(file_path, on_bad_lines="skip", encoding="utf-8")
    except pd.errors.ParserError as e:
        raise Exception(f"CSV 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")



In [3]:
def generate_summary(data: pd.DataFrame) -> pd.DataFrame:
    """
    生成數據描述性統計。
    Args:
        data (pd.DataFrame): 輸入數據框。
    Returns:
        pd.DataFrame: 描述性統計。
    """
    try:
        return data.describe()
    except Exception as e:
        raise Exception(f"生成描述性統計時發生錯誤：{e}")

In [4]:
def construct_prompt(user_input: str, summary_stats: pd.DataFrame) -> tuple:
    """
    構建系統與用戶提示。
    Args:
        user_input (str): 用戶查詢。
        summary_stats (pd.DataFrame): 描述性統計。
    Returns:
        tuple: (系統提示, 用戶提示)
    """
    system_prompt =( 
    
    """
        You are a professional data scientist skilled in extracting valuable insights from data. 
        Based on the provided descriptive statistics, perform an in-depth analysis and generate actionable recommendations in Chinese:
        1. Summarize the overall characteristics of the data, such as mean, standard deviation, and distribution.
        2. Identify potential outliers or data quality issues, and evaluate their impact on the analysis.
        3. If there are categorical variables, analyze their distribution and possible correlations with other features.
        4. Provide actionable recommendations based on the data, such as potential improvement areas or specific domains that require attention.
        5. Clearly and concisely communicate your conclusions, and suggest next steps for further data collection or analysis.

        The final report should be written in tranditional Chinese.
        """
    )

    user_prompt = (
        f"以下是數據的描述性統計結果：\n{summary_stats.to_string()}\n"
        f"請根據這些數據回答以下問題：{user_input}"
    )
    return system_prompt, user_prompt


In [5]:
def generate_report(file_path: str, user_input: str) -> str:
    """
    基於自然語言查詢生成分析報告。
    Args:
        file_path (str): 數據文件路徑。
        user_input (str): 用戶的自然語言查詢。
    Returns:
        str: 分析報告或錯誤信息。
    """
    try:
        # 1. 加載數據
        data = load_data(file_path)

        # 2. 生成描述性統計
        summary_stats = generate_summary(data)

        # 3. 構建 GPT 模型請求的提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)

        # 4. 初始化 GPT 客戶端並生成報告
        client = Client()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"



In [6]:
if __name__ == "__main__":
    # 示例使用
    file_path = r".\csv.ingredient\test.csv"


    user_query = "請分析數據並提供詳細的見解和建議。"  # 自然語言查詢

    try:
        report = generate_report(file_path, user_query)
        print("生成的報告：")
        print(report)
    except Exception as e:
        print(f"程序運行失敗：{e}")

New g4f version: 0.4.0.2 (current: 0.3.9.7) | pip install -U g4f
生成的報告：
### 數據分析報告

#### 1. 整體特徵總結
根據提供的描述性統計結果，我們可以觀察到以下幾個特徵：

- **年份 (Year)**：
  - 數據範圍從2010年到2024年，共60個樣本。
  - 平均年份為2017年，顯示數據主要集中在2010年至2024年之間。

- **市盈率 (P/E Ratio)**：
  - 平均市盈率為20.463，標準差為3.606，顯示市盈率的變化幅度相對較小。
  - 最小值為14.110，最大值為26.650，顯示市盈率的範圍相對廣泛。
  - 四分位數顯示，25%的數據低於17.658，50%的數據（中位數）為20.715，75%的數據高於23.863，這表明市盈率的分佈略偏右，可能存在一些高市盈率的異常值。

#### 2. 潛在異常值或數據質量問題
在市盈率的範圍內，最大值26.650與平均值20.463相比，顯示出可能存在異常值。這些異常值可能會影響整體分析結果，特別是在計算平均值和標準差時。建議進一步檢查這些高市盈率的數據點，確定其是否為數據輸入錯誤或真實的市場現象。

#### 3. 類別變數分析
在此數據集中，只有年份和市盈率兩個變數，並未提供其他類別變數的數據。如果有其他類別變數（如行業類別、地區等），可以進一步分析其分佈情況及其與市盈率的相關性。例如，不同行業的市盈率可能存在顯著差異，這可以幫助我們理解市場趨勢。

#### 4. 可行的建議
基於以上分析，以下是幾個可行的建議：

- **數據清理**：對於市盈率中的潛在異常值，建議進行進一步的數據清理，確保數據的準確性和可靠性。
- **擴展數據集**：考慮收集更多的類別變數數據，以便進行更深入的分析，特別是行業和地區的影響。
- **定期監測**：建立定期監測機制，跟踪市盈率的變化趨勢，及時發現市場異常情況。

#### 5. 結論與後續步驟
總結來說，該數據集提供了有價值的市場信息，但仍需進一步的數據清理和擴展。建議下一步進行以下操作：

- 檢查和處理市盈率中的異常值。
- 收集更多的類別變數數據，以便進行更全面的分析。
- 定期更新數據集，並持續監測市場變化。

這些步驟將有助於提高分析的準確